## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [1]:
import keras
from keras.datasets import cifar10
from resnet_builder import resnet # 這是從 resnet_builder.py 中直接 import 撰寫好的 resnet 函數
from keras.models import Model
from keras.optimizers import Adam
from keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [2]:
# 讀取資料集並作前處理
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train = x_train / 255.
x_test = x_test / 255.

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples


In [3]:
base_model = keras.applications.VGG16(input_shape=(32, 32, 3), include_top=False, weights='imagenet')

In [4]:
base_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 32, 32, 64)        1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 32, 32, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 16, 16, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 16, 16, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 16, 16, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 8, 8, 128)         0         
__________

In [5]:
#在捲積基底上增加密集層分類器
model = keras.models.Sequential()
model.add(base_model)
model.add(keras.layers.Flatten())
model.add(keras.layers.Dense(units=256, activation='relu'))
model.add(keras.layers.Dropout(0.5))
model.add(keras.layers.Dense(units=64, activation='relu'))
model.add(keras.layers.Dropout(0.3))
model.add(keras.layers.Dense(units=10, activation='softmax'))

In [6]:
#重要步驟: 要凍結預先訓練好的權重
print('number of trainable weights before freezing the conv base:', len(model.trainable_weights))
base_model.trainable = False
print('number of trainable weights after freezing the conv base:', len(model.trainable_weights))

number of trainable weights before freezing the conv base: 32
number of trainable weights after freezing the conv base: 6


In [7]:
train_datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

In [8]:
model.compile(loss='binary_crossentropy', optimizer=keras.optimizers.Adam(lr=3e-3), metrics=['acc'])

In [9]:
history = model.fit_generator(train_datagen.flow((x_train , y_train), batch_size=64), 
                    steps_per_epoch=1000,
                    epochs=10,
                    verbose=1,
                    validation_data=(x_test, y_test))

Epoch 1/10
1000/1000 [==============================] - 35s 35ms/step - loss: 0.2622 - acc: 0.9045 - val_loss: 0.2203 - val_acc: 0.9144
Epoch 2/10
1000/1000 [==============================] - 33s 33ms/step - loss: 0.2432 - acc: 0.9090 - val_loss: 0.2165 - val_acc: 0.9156
Epoch 3/10
1000/1000 [==============================] - 33s 33ms/step - loss: 0.2383 - acc: 0.9105 - val_loss: 0.2077 - val_acc: 0.9189
Epoch 4/10
1000/1000 [==============================] - 34s 34ms/step - loss: 0.2359 - acc: 0.9111 - val_loss: 0.2071 - val_acc: 0.9197
Epoch 5/10
1000/1000 [==============================] - 35s 35ms/step - loss: 0.2338 - acc: 0.9119 - val_loss: 0.2062 - val_acc: 0.9196
Epoch 6/10
1000/1000 [==============================] - 33s 33ms/step - loss: 0.2320 - acc: 0.9123 - val_loss: 0.2118 - val_acc: 0.9164
Epoch 7/10
1000/1000 [==============================] - 36s 36ms/step - loss: 0.2319 - acc: 0.9127 - val_loss: 0.2014 - val_acc: 0.9208
Epoch 8/10
1000/1000 [==========================